<a href="https://colab.research.google.com/github/ThuyHaLE/ThuyHaLE/blob/main/Dash_Plotly_Covid19_info_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TOOLs

In [ ]:
!pip install --upgrade plotly
!pip install jupyter-dash
!pip install dash-cytoscape

In [76]:
import pandas as pd
import numpy as np
import math
import datetime

In [77]:
import plotly.graph_objects as go
import plotly.express as px

In [78]:
from jupyter_dash import JupyterDash

import dash
from dash import no_update, dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output 
import dash_cytoscape as cyto

#DATA

##UPDATED DATA BY WHO (https://covid19.who.int)

In [82]:
#Open the file and read it into a variable

#1. Daily cases and deaths by date reported to WHO
daily_covid_19 = pd.read_csv(
    'https://covid19.who.int/WHO-COVID-19-global-data.csv')
daily_covid_19 = daily_covid_19.rename(columns = {'Country_code':'Alpha-2 code'})

#2. Latest reported counts of cases and deaths
count_covid_19 = pd.read_csv(
    'https://covid19.who.int/WHO-COVID-19-global-table-data.csv')
count_covid_19.index.names = ['Country']
count_covid_19.reset_index (inplace=True)
count_covid_19 = count_covid_19.rename(columns = {'Name':'Subregion'})
count_covid_19['Subregion'] = count_covid_19['Subregion'].fillna('Global')

In [83]:
#Take a look at the data (Latest reported counts of cases and deaths)
count_covid_19.head()

,Country,Subregion,WHO Region,Cases - cumulative total,Cases - cumulative total per 100000 population,Cases - newly reported in last 7 days,Cases - newly reported in last 7 days per 100000 population,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 100000 population,Deaths - newly reported in last 7 days,Deaths - newly reported in last 7 days per 100000 population,Deaths - newly reported in last 24 hours
0,Global,Global,199466211,2559.050333,4158340,53.349393,548167,4244541,54.455309,64036,0.821549,8430,NaN
1,United States of America,Americas,35010407,10577.080000,618994,187.010000,78722,609022,183.990000,2759,0.830000,387,NaN
2,India,South-East Asia,31769132,2302.100000,284527,20.620000,42625,425757,30.850000,3735,0.270000,562,NaN
3,Brazil,Americas,19953501,9387.260000,245839,115.660000,15143,557223,262.150000,6721,3.160000,389,NaN
4,Russian Federation,Europe,6356784,4355.920000,161552,110.700000,22589,161715,110.810000,5537,3.790000,790,NaN


In [84]:
#Take a look at the data (Daily cases and deaths by date reported to WHO)
daily_covid_19.head()

,Date_reported,Alpha-2 code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0


##INSERT MORE FEATURES FOR THE "Latest reported counts of cases and deaths"

First, we will use "Daily cases and deaths by date reported to WHO" to add a new column named 'Alpha_2 code' corresponding to each country.

Next, we will use the 'Alpha_2 code' and the country codes list of www.iban.com to add a new column named 'Alpha_3 code' corresponding to each country.

Finally, we will use 'The alpha_3 code and population list of the world from The World Bank to add a new column named 'Population 2021' corresponding to each country.

*Note that We will ignore countries that do not on the population list of The World Bank*

In [85]:
#ADD A NEW COLUMN NAMED ALPHA_2 CODE
ALPHA_2_CODE = daily_covid_19[daily_covid_19[['Alpha-2 code',	'Country']].duplicated() == False][['Alpha-2 code',	'Country']]
add_ALPHA_2 = count_covid_19[1:].merge(ALPHA_2_CODE, how='left', on = 'Country')

#ADD A NEW COLUMN NAMED ALPHA_3 CODE
country_code = pd.read_excel('https://drive.google.com/uc?export=download&id=131ohJe8iLbs2kG7lf2_fp7hKrV_De4UH') #Source: https://www.iban.com/country-codes
add_ALPHA_3 = add_ALPHA_2.merge(country_code[['Alpha-2 code',	'Alpha-3 code']], how = 'left', on = 'Alpha-2 code')
add_ALPHA_3

#ADD A NEW COLUMN NAMED POPULATION 2021
population = pd.read_csv('https://drive.google.com/uc?export=download&id=1lqeqNkqUp1EppiYuikC0wDV6E6AIhEGk') #Source: World Development Indicators (Last Updated Date: 9/16/2022)
population = population.rename(columns = {'Country Code':'Alpha-3 code','Country Name':'Country'})
add_population = add_ALPHA_3.merge(population[['Alpha-3 code','2021']], how = 'left', on = ['Alpha-3 code'])
count_covid_19_updated = add_population[add_population['2021'].isnull()==False]
count_covid_19_updated = count_covid_19_updated.rename(columns = {'2021':'Population 2021'})
count_covid_19_updated

,Country,Subregion,WHO Region,Cases - cumulative total,Cases - cumulative total per 100000 population,Cases - newly reported in last 7 days,Cases - newly reported in last 7 days per 100000 population,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 100000 population,Deaths - newly reported in last 7 days,Deaths - newly reported in last 7 days per 100000 population,Deaths - newly reported in last 24 hours,Alpha-2 code,Alpha-3 code,Population 2021
0,United States of America,Americas,35010407,10577.08,618994,187.01,78722,609022,183.99,2759,0.83,387,NaN,US,USA,3.318937e+08
1,India,South-East Asia,31769132,2302.10,284527,20.62,42625,425757,30.85,3735,0.27,562,NaN,IN,IND,1.393409e+09
2,Brazil,Americas,19953501,9387.26,245839,115.66,15143,557223,262.15,6721,3.16,389,NaN,BR,BRA,2.139934e+08
3,Russian Federation,Europe,6356784,4355.92,161552,110.70,22589,161715,110.81,5537,3.79,790,NaN,RU,RUS,1.434461e+08
4,France,Europe,6039483,9285.89,139087,213.85,25481,110921,170.54,239,0.37,62,NaN,FR,FRA,6.749934e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,Nauru,Western Pacific,0,0.00,0,0.00,0,0,0.00,0,0.00,0,NaN,NR,NRU,1.087300e+04
230,Palau,Western Pacific,0,0.00,0,0.00,0,0,0.00,0,0.00,0,NaN,PW,PLW,1.817400e+04
234,Tonga,Western Pacific,0,0.00,0,0.00,0,0,0.00,0,0.00,0,NaN,TO,TON,1.067590e+05
235,Turkmenistan,Europe,0,0.00,0,0.00,0,0,0.00,0,0.00,0,NaN,TM,TKM,6.117933e+06


#VISUALIZATION

##FOR THE "Latest reported counts of cases and deaths"

In [86]:
#GET A COPY OF DATA
df = count_covid_19_updated.copy().fillna(0)

In [87]:
data = df.copy()
data['Deaths/Cases - cumulative total'] = data['Deaths - cumulative total']/data['Cases - cumulative total']
data['Cases/population'] = data['Cases - cumulative total']/data['Population 2021']
data['Deaths/population'] = data['Deaths - cumulative total']/data['Population 2021']

data['Deaths/Cases - cumulative total'] = data['Deaths/Cases - cumulative total'].fillna(0)
data['Cases/population'] = data['Cases/population'].fillna(0)
data['Deaths/population'] = data['Deaths/population'].fillna(0)

###BAR CHART


####Death To Case Ratio (%) by Country and Subregion

In [88]:
#Death To Case Ratio (%) by Country and Subregion
fig = px.bar(data.sort_values(by = ['Deaths/Cases - cumulative total'],axis = 0, ascending = True), 
             x="Deaths/Cases - cumulative total", 
             y="Country", text_auto='.3%',
             orientation='h',
             hover_name='Country', 
             hover_data=['Deaths - cumulative total', 'Cases - cumulative total'],
             color='Subregion',
             height = 3000,
             title='Death To Case Ratio (%) by Country and Subregion'
             ) 
fig.update_xaxes(#tickangle=45, 
                 tickfont=dict(family='Rockwell', color='crimson', size=10), showticklabels=True)
fig.update_yaxes(#tickangle=45, 
                 tickfont=dict(family='Rockwell', color='crimson', size=10), showticklabels=True)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)

fig.update_layout(
    font=dict(
        family='Rockwell',
        size=8,  # Set the font size here
        color='crimson'
    )
)
fig.show()

In [104]:
app = JupyterDash(__name__)

colors = {'background': '#111111','text': '#7FDBFF'}

app.layout = html.Div([
    html.H4('Death To Case Ratio (%) by Country and Subregion',
            style={'textAlign': 'center', 'color': colors['text'], 'font-size': 40}),
            html.P('There are six subregions. Please choose one to show the result.',
                   style={'textAlign':'center', 'color': '#F57241', 'font-size': 20}),

    dcc.Dropdown(id="dropdown",
        options=['Americas', 'South-East Asia', 'Europe', 'Western Pacific', 'Eastern Mediterranean', 'Africa'],
        value='Americas',clearable=False,),

    html.Br(),

    dcc.Graph(id="graph"),
])

@app.callback(
    Output("graph", "figure"), 
    Input("dropdown", "value"))

def display_Subregion(Subregion):
    df = data.sort_values(by = ['Deaths/Cases - cumulative total'],axis = 0, ascending = True)
    mask = df["Subregion"] == Subregion
    fig = px.bar(df[mask], 
             x="Deaths/Cases - cumulative total", y="Country", text_auto='.3%',
             orientation='h',hover_name='Country',
             hover_data=['Deaths - cumulative total', 'Cases - cumulative total'],color='Subregion',
             height = 700,
             title='Death To Case Ratio (%) by Country and Subregion'
             ) 
    fig.update_xaxes(tickfont=dict(family='Rockwell', color='crimson', size=10), showticklabels=False)
    fig.update_yaxes(tickfont=dict(family='Rockwell', color='crimson', size=7), showticklabels=True)
    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_layout(font=dict(family='Rockwell',size=10,color='crimson'))
    return fig

app.run_server(mode="inline")

<IPython.core.display.Javascript object>

####Death To Case Ratio (%) by Country

In [90]:
#Death To Case Ratio (%) by Country
fig = px.bar(data.sort_values(by = ['Deaths/Cases - cumulative total'],axis = 0, ascending = False), 
             x="Deaths/Cases - cumulative total", 
             y="Country", text_auto='.3%',
             orientation='h',
             hover_name='Country',
             hover_data=['Subregion','Deaths - cumulative total', 'Cases - cumulative total'],
             #color='Subregion',
             height = 3000,
             title='Death To Case Ratio (%) by Country'
             ) 
fig.update_xaxes(#tickangle=45, 
                 tickfont=dict(family='Rockwell', color='crimson', size=10), showticklabels=True)
fig.update_yaxes(#tickangle=45, 
                 tickfont=dict(family='Rockwell', color='crimson', size=10), showticklabels=True)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)

fig.update_layout(
    font=dict(
        family='Rockwell',
        size=8,  # Set the font size here
        color='crimson'
    )
)
fig.show()

In [105]:
app = JupyterDash(__name__)

colors = {'background': '#111111','text': '#7FDBFF'}

app.layout = html.Div([
    html.H4('Death To Case Ratio (%) by Country',
            style={'textAlign': 'center', 'color': colors['text'], 'font-size': 40}),
            html.P('Please choose the top highest rated countries.',
                   style={'textAlign':'center', 'color': '#F57241', 'font-size': 20}),

    html.Div([dcc.Dropdown(id="Dropdown", options=[
        {"label": "ALL", "value": 'ALL'},
        {"label": "TOP 30", "value": 30},
        {"label": "TOP 20", "value": 20},
        {"label": "TOP 15", "value": 15},
        {"label": "TOP 10", "value": 10},
        {"label": "TOP 5", "value": 5},
        {"label": "TOP 3", "value": 3}],
        value=5, placeholder="Select a top", clearable=False, maxHeight=300),
        html.Div(id='pandas-output-container-2',style={'color': 'Gold', 'font-size': 18, 'padding-left': 10})
        ]),
    
    html.Br(),
    html.Br(), 

    dcc.Graph(id="graph")],)    

@app.callback(
    Output('pandas-output-container-2', 'children'),
    Input('Dropdown', 'value'))
def update_output(value):
    return f'You have selected {value} highest rated countries.'

@app.callback(
    Output("graph", "figure"),
    Input("Dropdown", "value"))
def display_top_country(value):
    df = data.sort_values(by = ['Deaths/Cases - cumulative total'],axis = 0, ascending = False)
    
    if value == 'ALL':
      fig = px.bar(df.head(len(df)),
              x="Deaths/Cases - cumulative total", y="Country", text_auto='.3%',
              orientation='h',
              hover_name='Country',
              hover_data=['Subregion','Deaths - cumulative total', 'Cases - cumulative total'],
              height = 3000,
              title='Death To Case Ratio (%) by Country and Subregion'
              ) 
      fig.update_xaxes(tickfont=dict(family='Rockwell', color='crimson', size=10), showticklabels=False)
      fig.update_yaxes(tickfont=dict(family='Rockwell', color='crimson', size=7), showticklabels=True)
      fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
      fig.update_layout(font=dict(family='Rockwell',size=10,color='crimson'))
      return fig
    else:
      fig = px.bar(df.head(value),
              x="Deaths/Cases - cumulative total", y="Country", text_auto='.3%',
              orientation='h',
              hover_name='Country',
              hover_data=['Subregion','Deaths - cumulative total', 'Cases - cumulative total'],
              height = 600,
              title='Death To Case Ratio (%) by Country and Subregion'
              ) 
      fig.update_xaxes(tickfont=dict(family='Rockwell', color='crimson', size=10), showticklabels=False)
      fig.update_yaxes(tickfont=dict(family='Rockwell', color='crimson', size=10), showticklabels=True)
      fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
      fig.update_layout(font=dict(family='Rockwell',size=10,color='crimson'))
      return fig

app.run_server(mode="inline")

<IPython.core.display.Javascript object>

####Morbidity and mortality by Subregion

In [92]:
#Morbidity and mortality by Subregion
data_1 = data.groupby('Subregion')[['Deaths - cumulative total', 'Cases - cumulative total']].sum().reset_index()
fig = go.Figure(data=[
    go.Bar(name='Cases - cumulative total', x=data_1["Subregion"], y=data_1['Cases - cumulative total'],text=round(data_1['Cases - cumulative total'],2),
            textposition="outside"),
    go.Bar(name='Deaths - cumulative total', x=data_1["Subregion"], y=data_1['Deaths - cumulative total'],text=round(data_1['Deaths - cumulative total'],2),
            textposition="outside")
])

fig.update_xaxes(#tickangle=45, 
                 tickfont=dict(family='Rockwell', color='crimson', size=10), showticklabels=True)
fig.update_yaxes(#tickangle=45, 
                 tickfont=dict(family='Rockwell', color='crimson', size=10), showticklabels=True)

fig.update_layout(
    font=dict(family='Rockwell', size=10, color='crimson'),
        barmode='group', 
        #height = 1000,
        title='Morbidity and mortality by Subregion', 
        xaxis_title="Subregion",
        yaxis_title="Cumulative total")
fig.show()

In [93]:
#Mortality by Subregion
fig = px.bar(data, 
             y=['Deaths - cumulative total'], 
             x="Subregion", 
             #text_auto='.3',
             #orientation='h',
             hover_name='Country',
             #hover_data=[, 'Cases - cumulative total'],
             color='Subregion',
             pattern_shape="Subregion", 
             pattern_shape_sequence=[".", "x", "+"],
             barmode = 'stack',
             #height = 3000,
             title='Mortality by Subregion'
             ) 

fig.add_hline(y=(data.groupby("Subregion")['Deaths - cumulative total'].sum()).mean(), 
              annotation_text="Deaths - average by Subregion", 
              annotation_position="top right",
              line_width=2,
              line_color='Red',
              yref='paper')

fig.update_layout(
        #xaxis_title="Subregion",
        yaxis_title="Mortality cumulative total")

fig.show()

In [94]:
#Morbidity by Subregion
fig = px.bar(data, 
             y=['Cases - cumulative total'], 
             x="Subregion", 
             #text_auto='3',
             #orientation='h',
             hover_name='Country',
             #hover_data=[, 'Cases - cumulative total'],
             color='Subregion',
             pattern_shape="Subregion", 
             pattern_shape_sequence=[".", "x", "+"],
             barmode = 'stack',
             #height = 3000,
             title='Morbidity by Subregion'
             ) 

fig.add_hline(y=(data.groupby("Subregion")['Cases - cumulative total'].sum()).mean(), 
              annotation_text="Cases - average by Subregion", 
              annotation_position="top right",
              line_width=2,
              line_color='Red',
              yref='paper')

fig.update_layout(
        #xaxis_title="Subregion",
        yaxis_title="Morbidity cumulative total")

fig.show()

In [95]:
#Death To Case Ratio (%) by Subregion
data_2 = data.copy().groupby('Subregion')[['Cases - cumulative total','Deaths - cumulative total']].sum().reset_index()
data_2['Deaths/Cases - cumulative total'] = data_2['Deaths - cumulative total']/data_2['Cases - cumulative total']
data_2 = data_2.sort_values(by = ['Deaths/Cases - cumulative total'], axis = 0, ascending = True)

fig = px.bar(data_2, 
             x="Subregion", y="Deaths/Cases - cumulative total",text_auto='.3%', 
             text="Deaths/Cases - cumulative total",
             #orientation='h',
             #hover_name='Cases - cumulative total',
             hover_data=['Deaths - cumulative total', 'Cases - cumulative total'],
             #height = 3000,
             title='Death To Case Ratio (%) by Subregion') 

#fig.update_xaxes(showticklabels=False) # Hide x axis ticks 
fig.update_yaxes(showticklabels=False) # Hide y axis ticks

fig.update_layout(
    font=dict(
        family="Times New Roman, monospace",
        size=12,  # Set the font size here
        color="Blue"
    )
)
fig.show()

In [106]:
app = JupyterDash(__name__)

colors = {'background': '#111111','text': '#7FDBFF'}

app.layout = html.Div([
    html.H4('Morbidity and mortality by Subregion',
            style={'textAlign': 'center', 'color': colors['text'], 'font-size': 40}),
            html.P('Please choose the option.',
                   style={'textAlign':'center', 'color': '#F57241', 'font-size': 20}),

    html.Div([dcc.Dropdown(id="Dropdown", options=["Morbidity and mortality by Subregion",
                                                   "Mortality by Subregion", 
                                                   "Morbidity by Subregion",
                                                   "Death To Case Ratio (%) by Subregion"],
                           value="Morbidity and mortality by Subregion", 
                           clearable=False, maxHeight=300),
              html.Div(id='pandas-output-container-2',style={'color': 'Gold', 'font-size': 18, 'padding-left': 10})
              ]),
    
    html.Br(),
    html.Br(), 

    dcc.Graph(id="graph")],)    

@app.callback(
    Output('pandas-output-container-2', 'children'),
    Input('Dropdown', 'value'))
def update_output(value):
    return f'You have selected {value}.'

@app.callback(
    Output("graph", "figure"),
    Input("Dropdown", "value"))
def display_top_country(value):
    df = data.copy()
    if value == 'Morbidity and mortality by Subregion':
      data_1 = df.groupby('Subregion')[['Deaths - cumulative total', 'Cases - cumulative total']].sum().reset_index()
      fig = go.Figure(data=[
          go.Bar(name='Cases - cumulative total', x=data_1["Subregion"], 
                 y=data_1['Cases - cumulative total'],text=round(data_1['Cases - cumulative total'],2),
                 textposition="outside"),
          go.Bar(name='Deaths - cumulative total', x=data_1["Subregion"], 
                 y=data_1['Deaths - cumulative total'],text=round(data_1['Deaths - cumulative total'],2),
                 textposition="outside")
          ])
      fig.update_xaxes(tickfont=dict(family='Rockwell', color='crimson', size=10), showticklabels=True)
      fig.update_yaxes(tickfont=dict(family='Rockwell', color='crimson', size=10), showticklabels=True)
      fig.update_layout(font=dict(family='Rockwell', size=10, color='crimson'),
                        barmode='group', title='Morbidity and mortality by Subregion', 
                        xaxis_title="Subregion",yaxis_title="Cumulative total")
      return fig  
    elif value == "Death To Case Ratio (%) by Subregion":
      data_2 = df.copy().groupby('Subregion')[['Cases - cumulative total','Deaths - cumulative total']].sum().reset_index()
      data_2['Deaths/Cases - cumulative total'] = data_2['Deaths - cumulative total']/data_2['Cases - cumulative total']
      data_2 = data_2.sort_values(by = ['Deaths/Cases - cumulative total'], axis = 0, ascending = True)
      fig = px.bar(data_2, x="Subregion", y="Deaths/Cases - cumulative total",text_auto='.3%', color='Subregion',
             text="Deaths/Cases - cumulative total",hover_data=['Deaths - cumulative total', 'Cases - cumulative total'],
             title='Death To Case Ratio (%) by Subregion') 
      fig.update_yaxes(showticklabels=False)
      fig.update_layout(font=dict(family="Times New Roman, monospace", size=12,  color="Blue"))
      return fig
    elif value == "Mortality by Subregion":
      fig = px.bar(df, y=['Deaths - cumulative total'], x="Subregion", hover_name='Country',
             color='Subregion',pattern_shape="Subregion", pattern_shape_sequence=[".", "x", "+"], 
             barmode = 'stack',title='Mortality by Subregion'
             ) 
      fig.add_hline(y=(df.groupby("Subregion")['Deaths - cumulative total'].sum()).mean(), 
                    annotation_text="Deaths - average by Subregion", annotation_position="top right",
                    line_width=2,line_color='Red',yref='paper')
      fig.update_layout(yaxis_title="Mortality cumulative total")
      return fig
    elif value == "Morbidity by Subregion":
      fig = px.bar(df, y=['Cases - cumulative total'], x="Subregion", hover_name='Country',
                   color='Subregion',pattern_shape="Subregion", pattern_shape_sequence=[".", "x", "+"],
                   barmode = 'stack',title='Morbidity by Subregion'
                   ) 
      fig.add_hline(y=(df.groupby("Subregion")['Cases - cumulative total'].sum()).mean(), 
                    annotation_text="Cases - average by Subregion", annotation_position="top right",
                    line_width=2,line_color='Red',yref='paper')
      fig.update_layout(yaxis_title="Morbidity cumulative total")
      return fig

app.run_server(mode="inline")

<IPython.core.display.Javascript object>

###SCATTER

####Death To Case Ratio by country

In [97]:
#Death To Case Ratio by country
fig = px.scatter(data, x='Cases - cumulative total', y="Deaths - cumulative total", 
                 #text_auto='.3%',
                 size="Deaths/Cases - cumulative total",color='Subregion', 
                 hover_data = ['Cases - cumulative total', "Deaths - cumulative total", 
                               "Deaths/Cases - cumulative total"],
                 hover_name="Country",size_max=120,title='Death To Case Ratio by country')

fig.add_hline(y=data['Deaths - cumulative total'].mean(), 
              annotation_text="Deaths - average", annotation_position="bottom right",
              line_width=2, line_dash='dash',line_color='Red',yref='paper')

fig.add_vline(x=data['Cases - cumulative total'].mean(), 
              annotation_text= 'Cases - average', annotation_position="top right",
              line_width=3, line_dash="dot", line_color="green")

fig.show()

In [107]:
app = JupyterDash(__name__)
colors = {'background': '#111111','text': '#7FDBFF'}

app.layout = html.Div([
    html.H4('Death To Case Ratio by country',
            style={'textAlign': 'center', 'color': colors['text'], 'font-size': 40}),
    html.Br(),
    dcc.Store(id='intermediate-value'),
    html.Br(),
    dcc.Dropdown(options=[
        {'label': 'Deaths/Cases - cumulative total sort by descending', 'value': 'descending'},
        {'label': 'Deaths/Cases - cumulative total sort by ascending', 'value': 'ascending'}], 
        value='ascending',id='dropdown'),
        html.Div(id='dd-output-container',style={'color': 'white', 'font-size': 12, 'padding-left': 10}),
    html.Br(),
    dcc.Slider(1, 209, value=20, id='my-slider',marks={
        5: {'label': 'Top5', 'style': {'color': '#77b0b1'}},
        50: {'label': 'Top50', 'style': {'color': '#77b0b1'}},
        100: {'label': 'Top100', 'style': {'color': '#77b0b1'}},
        209: {'label': 'All', 'style': {'color': '#f50'}}},included=False),
        html.Div(id='slider-output-container',style={'color': 'Gold', 'font-size': 12, 'padding-left': 10}),
    html.Br(),
    dcc.Graph(id="graph"),
        ])

@app.callback(
    Output('dd-output-container', 'children'),
    Input('dropdown', 'value'))
def update_output(value):
  return 'You have selected Deaths/Cases - cumulative total sort by {}'.format(value)

@app.callback(
    Output('intermediate-value', 'data'),
    Input('dropdown', 'value'))
def clean_data(value):
  if value == 'descending':
    cleaned_df = data[data['Deaths/Cases - cumulative total']>0].copy().sort_values(by = ['Deaths/Cases - cumulative total'], ascending = False)
    return cleaned_df.to_json(date_format='iso', orient='split')
  elif value == 'ascending':
    cleaned_df = data[data['Deaths/Cases - cumulative total']>0].copy().sort_values(by = ['Deaths/Cases - cumulative total'], ascending = True)
    return cleaned_df.to_json(date_format='iso', orient='split')

@app.callback(
    Output('slider-output-container', 'children'),
    Input('my-slider', 'value'))
def update_output(value):
  return 'You have selected top {} countries'.format(value)

@app.callback(Output("graph", "figure"),
              Input('intermediate-value', 'data'),
              Input("my-slider", "value"))
def update_graph(jsonified_cleaned_data, value):
  df = pd.read_json(jsonified_cleaned_data, orient='split').head(value)
  fig = px.scatter(df, x='Cases - cumulative total', y="Deaths - cumulative total",hover_name="Country", size_max=120,
                   hover_data = ['Cases - cumulative total', "Deaths - cumulative total", "Deaths/Cases - cumulative total"],
                   color='Subregion', size="Deaths/Cases - cumulative total", title='Death To Case Ratio by country')
  fig.add_hline(y=df['Deaths - cumulative total'].mean(), 
                annotation_text="Deaths - average", annotation_position="bottom right",
                line_width=2, line_dash='dash',line_color='Red',yref='paper')
  fig.add_vline(x=df['Cases - cumulative total'].mean(), 
                annotation_text= 'Cases - average', annotation_position="top right",
                line_width=3, line_dash="dot", line_color="green")
  return fig

app.run_server(mode="inline")

<IPython.core.display.Javascript object>

####Morbidity vs mortality by country

In [99]:
#Morbidity vs mortality by country
#Import make_subplots function from plotly.subplots to make grid of plots
from plotly.subplots import make_subplots

#Use specs parameter in make_subplots function to create secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
 
#Plot a scatter chart by specifying the x and y values and use add_trace function to specify secondary_y axes
fig.add_trace(go.Scatter(x=data["Country"], y=data['Cases - cumulative total'],
                         mode='lines+markers'),secondary_y=False)
fig.update_traces(name='Morbidity by country', showlegend = True,secondary_y=False)
fig.add_trace(go.Scatter(x=data["Country"], y=data['Deaths - cumulative total'],
                         mode='lines+markers'),secondary_y=True)
fig.update_traces(name='Mortality by country', showlegend = True,secondary_y=True)

#Adding title text to the figure
fig.update_layout(title_text="Morbidity vs mortality by country")

#Naming x-axis
fig.update_xaxes(tickangle=45, title_text="Country",color='black')

#Naming y-axes
fig.update_yaxes(title_text="Cases - cumulative total", color='blue', secondary_y=False)
fig.update_yaxes(title_text="Deaths - cumulative total", color='red', secondary_y=True)

fig.add_shape(type="line",xref="paper", yref="y1",x0=0, y0=data['Cases - cumulative total'].mean(), 
              x1=0.94, y1=data['Cases - cumulative total'].mean(),line=dict(color="blue"),)
fig.add_annotation(x=-10, y=data['Cases - cumulative total'].mean()-1500,text="Cases - cumulative average", 
                   showarrow=False,font=dict(color="blue",size=10),)

fig.add_shape(type="line",xref="paper", yref="y2",x0=0, y0=data['Deaths - cumulative total'].mean(), 
              x1=0.94, y1=data['Deaths - cumulative total'].mean(),line=dict(color="red"),)
fig.add_annotation(x=220, y=data['Deaths - cumulative total'].mean()-15,text="Deaths - cumulative average",
                   xref="x", yref="y2", showarrow=False,font=dict(color="red",size=10),)

fig.update_layout(font=dict(family='Rockwell', size=8, color='crimson'),width = 3000)
fig.show()

In [108]:
app = JupyterDash(__name__)
colors = {'background': '#111111','text': '#7FDBFF'}

app.layout = html.Div([
    html.H4('Morbidity vs mortality by country',
            style={'textAlign': 'center', 'color': colors['text'], 'font-size': 40}),
    html.Br(),
    dcc.Slider(1, 212, value=20, id='my-slider',marks={
        5: {'label': 'Top5', 'style': {'color': '#77b0b1'}},
        50: {'label': 'Top50', 'style': {'color': '#77b0b1'}},
        100: {'label': 'Top100', 'style': {'color': '#77b0b1'}},
        212: {'label': 'All', 'style': {'color': '#f50'}}},included=False),
        html.Div(id='slider-output-container',style={'color': 'Gold', 'font-size': 12, 'padding-left': 10}),
    html.Br(),
    dcc.Graph(id="graph")
            ])

@app.callback(
    Output('slider-output-container', 'children'),
    Input('my-slider', 'value'))
def update_output(value):
  return 'You have selected Cases/population top {}'.format(value)

@app.callback(Output("graph", "figure"),
              Input("my-slider", "value"))
def update_graph(value):
  df = data.copy().head(value)
  from plotly.subplots import make_subplots
  fig = make_subplots(specs=[[{"secondary_y": True}]])
  fig.add_trace(go.Scatter(x=df["Country"], y=df['Cases - cumulative total'],mode='lines+markers'),secondary_y=False)
  fig.update_traces(name='Morbidity by country', showlegend = True,secondary_y=False)
  fig.add_trace(go.Scatter(x=df["Country"], y=df['Deaths - cumulative total'],mode='lines+markers'),secondary_y=True)
  fig.update_traces(name='Mortality by country', showlegend = True,secondary_y=True)
  fig.update_layout(title_text="Morbidity vs mortality by country")
  fig.update_xaxes(tickangle=45, title_text="Country",color='black')
  fig.update_yaxes(title_text="Cases - cumulative total", color='blue', secondary_y=False)
  fig.update_yaxes(title_text="Deaths - cumulative total", color='red', secondary_y=True)

  fig.add_shape(type="line",xref="paper", yref="y1",x0=0, y0=df['Cases - cumulative total'].mean(),
                x1=0.94, y1=df['Cases - cumulative total'].mean(),line=dict(color="blue"),)
  fig.add_annotation(x=-3, y=df['Cases - cumulative total'].mean()-1500,
                     text="Cases - cumulative average", showarrow=False,font=dict(color="blue",size=8),)
  
  fig.add_shape(type="line",xref="paper", yref="y2",x0=0, y0=df['Deaths - cumulative total'].mean(),
                x1=0.94, y1=df['Deaths - cumulative total'].mean(),line=dict(color="red"))
  fig.add_annotation(x=-3, y=df['Deaths - cumulative total'].mean()-10,
                     xref="x",yref="y2",text="Deaths - cumulative average", showarrow=False,font=dict(color="red",size=8),)
  
  fig.update_layout(font=dict(family='Rockwell', size=8, color='crimson'),
                    width = 1200
                    )
  return fig

app.run_server(mode="inline")

<IPython.core.display.Javascript object>

####Morbidity/population vs mortality/population by country

In [101]:
#Morbidity/population vs mortality/population by country
#Import make_subplots function from plotly.subplots to make grid of plots
from plotly.subplots import make_subplots

#Use specs parameter in make_subplots function to create secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
 
#Plot a scatter chart by specifying the x and y values and use add_trace function to specify secondary_y axes
fig.add_trace(go.Scatter(x=data["Country"], y=data['Cases/population'],mode='lines+markers'),secondary_y=False)
fig.update_traces(name='Cases/population by country', showlegend = True,secondary_y=False)
fig.add_trace(go.Scatter(x=data["Country"], y=data['Deaths/population'],mode='lines+markers'),secondary_y=True)
fig.update_traces(name='Deaths/population by country', showlegend = True,secondary_y=True)

#Adding title text to the figure
fig.update_layout(title_text="Morbidity/population vs mortality/population by country")

#Naming x-axis
fig.update_xaxes(tickangle=45, title_text="Country",color='black')

#Naming y-axes
fig.update_yaxes(title_text="Cases/population",color='blue', secondary_y=False)
fig.update_yaxes(title_text="Deaths/population",color='red', secondary_y=True)

fig.add_hrect(y0=1, y1=4.2, line_width=0.1, fillcolor="red", annotation_text="Over 100% zone (cases/population)", 
              annotation_position="bottom left", annotation_font_color="blue", 
              annotation_font_size=10, annotation_font_family="Rockwell",
              opacity=0.2, secondary_y=False)
fig.add_hrect(y0=0.0165, y1=0.07, line_width=0.1, fillcolor="red", opacity=0.2, secondary_y=True)

fig.update_layout(font=dict(family='Rockwell', size=8, color='crimson'),width = 3000)
fig.show()

In [109]:
app = JupyterDash(__name__)
colors = {'background': '#111111','text': '#7FDBFF'}
tabs_styles = {'height': '30px'}
tab_style = {'borderBottom': '1px solid #d6d6d6',
             'padding': '6px'}
tab_selected_style = {'borderTop': '1px solid #d6d6d6',
                      'borderBottom': '1px solid #d6d6d6',
                      'backgroundColor': '#119DFF',
                      'fontWeight': 'bold',
                      'color': 'blue','padding': '6px'}

app.layout = html.Div([
    html.H4('Morbidity/population vs mortality/population by country',
            style={'textAlign': 'center', 'color': colors['text'], 'font-size': 40}),
    html.Br(),
    dcc.Store(id='intermediate-value'),
    html.Br(),
    dcc.Tabs(id='tabs-1', value='Cases/population', children=[
        dcc.Tab(label='Cases/population by country', value='Cases/population', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Deaths/population by country', value='Deaths/population', style=tab_style, selected_style=tab_selected_style)]),
        html.Div(id='tabs-content-1',style={'color': 'Gold', 'font-size': 12, 'padding-left': 10}),
    html.Br(),
    dcc.Slider(1, 212, value=20, id='my-slider',marks={
        5: {'label': 'Top5', 'style': {'color': '#77b0b1'}},
        50: {'label': 'Top50', 'style': {'color': '#77b0b1'}},
        100: {'label': 'Top100', 'style': {'color': '#77b0b1'}},
        212: {'label': 'All', 'style': {'color': '#f50'}}},included=False),
        html.Div(id='slider-output-container',style={'color': 'Gold', 'font-size': 12, 'padding-left': 10}),
    html.Br(),
    dcc.Graph(id="graph"),
        ])

@app.callback(
    Output('intermediate-value', 'data'),
    Input('tabs-1', 'value'))
def clean_data(value):
  cleaned_df = data.copy().sort_values(by = [value], ascending = False)
  return cleaned_df.to_json(date_format='iso', orient='split')

@app.callback(
    Output('tabs-content-1', 'children'),
    Input('tabs-1', 'value'))
def update_output(value):
  return 'You have selected {} by country'.format(value)

@app.callback(
    Output('slider-output-container', 'children'),
    Input('my-slider', 'value'))
def update_output(value):
  return 'You have selected top {} countries'.format(value)

@app.callback(Output("graph", "figure"),
              Input('intermediate-value', 'data'),
              Input("my-slider", "value"))
def update_graph(jsonified_cleaned_data, value):
  df = pd.read_json(jsonified_cleaned_data, orient='split').head(value)
  from plotly.subplots import make_subplots
  fig = make_subplots(specs=[[{"secondary_y": True}]])
  fig.add_trace(go.Scatter(x=df["Country"], y=df['Cases/population'],mode='lines+markers'),secondary_y=False)
  fig.update_traces(name='Cases/population by country', showlegend = True,secondary_y=False)
  fig.add_trace(go.Scatter(x=df["Country"], y=df['Deaths/population'],mode='lines+markers'),secondary_y=True)
  fig.update_traces(name='Deaths/population by country', showlegend = True,secondary_y=True)
  fig.update_layout(title_text="Morbidity/population vs mortality/population by country")
  fig.update_xaxes(tickangle=45, title_text="Country",color='black')
  fig.update_yaxes(title_text="Cases/population",color='blue', secondary_y=False)
  fig.update_yaxes(title_text="Deaths/population",range=[0, 0.07], color='red', secondary_y=True)
  fig.add_hrect(y0=1, y1=4, line_width=0.1, fillcolor="red", annotation_text="Over 100% zone (cases/population)", 
                annotation_position="bottom left", annotation_font_color="blue", 
                annotation_font_size=10, annotation_font_family="Rockwell",
                opacity=0.2, secondary_y=False)
  fig.update_layout(font=dict(family='Rockwell', size=8, color='crimson'),
                    width = 1200
                    )
  return fig

app.run_server(mode="inline")

<IPython.core.display.Javascript object>

###TABLE

In [110]:
df = data[['Subregion','Country','Cases - cumulative total','Deaths - cumulative total',
           'Population 2021','Deaths/Cases - cumulative total','Cases/population','Deaths/population']].round(2)
app = JupyterDash(__name__)
colors = {'background': '#111111','text': '#7FDBFF'}
app.layout = html.Div([
    html.H4('Latest reported counts of cases and deaths',style={'textAlign': 'center', 'color': colors['text'], 'font-size': 40}),
    dash_table.DataTable(id='table-filtering-be',
                         columns=[{'name': 'Subregion', 'id': 'Subregion', 'type': 'text'},
                                  {'name': 'Country', 'id': 'Country', 'type': 'text'},
                                  {'name': 'Cases - cumulative total', 'id': 'Cases - cumulative total', 'type': 'numeric'},
                                  {'name': 'Deaths - cumulative total', 'id': 'Deaths - cumulative total', 'type': 'numeric'},
                                  {'name': 'Population 2021', 'id': 'Population 2021', 'type': 'numeric'},
                                  {'name': 'Deaths/Cases - cumulative total', 'id': 'Deaths/Cases - cumulative total', 'type': 'numeric'},
                                  {'name': 'Cases/population', 'id': 'Cases/population', 'type': 'numeric'},
                                  {'name': 'Deaths/population', 'id': 'Deaths/population', 'type': 'numeric'}],
                         data=df.to_dict('records'),
                         page_size=10, 
                         page_action="native",
                         editable=True,
                         filter_action='custom',
                         filter_query='',
                         sort_action="native",
                         sort_mode="multi",
                         style_as_list_view=True,
                         fixed_rows={'headers': True},
                         style_table={'height': '300px', 'overflowY': 'auto'},
                         style_cell={'height': 'auto',
                                     'minWidth': '40px', 'width': '60px', 'maxWidth': '80px',
                                     'whiteSpace': 'normal'},
                         style_header={'font_family': 'Rockwell','font_size': '10px',
                                       'color': 'black','backgroundColor': 'white','fontWeight': 'bold',
                                       'border': '1px solid black',
                                       'whiteSpace': 'normal','height': 'auto',
                                       'text_align': 'center','padding': '5px'},
                         style_data={'font_family': 'Rockwell','font_size': '8px',
                                     'color': 'blue','backgroundColor': 'white',
                                     'whiteSpace': 'normal','height': 'auto'},),
    html.P('*For columns with numeric data, you can filter using the operators (>=, <=, <, >, !=, =).',
           style={'textAlign':'left', 'color': 'gold', 'font-size': 11}),])

operators = [['ge ', '>='],['le ', '<='],['lt ', '<'],
             ['gt ', '>'],['ne ', '!='],['eq ', '='],
             ['contains '],['datestartswith ']]

def split_filter_part(filter_part):
    for operator_type in operators:
        for operator in operator_type:
            if operator in filter_part:
                name_part, value_part = filter_part.split(operator, 1)
                name = name_part[name_part.find('{') + 1: name_part.rfind('}')]
                value_part = value_part.strip()
                v0 = value_part[0]
                if (v0 == value_part[-1] and v0 in ("'", '"', '`')):
                    value = value_part[1: -1].replace('\\' + v0, v0)
                else:
                    try: value = float(value_part)
                    except ValueError: value = value_part
                return name, operator_type[0].strip(), value
    return [None] * 3

@app.callback(
    Output('table-filtering-be', "data"),
    Input('table-filtering-be', "filter_query"))
def update_table(filter):
    filtering_expressions = filter.split(' && ')
    dff = df
    for filter_part in filtering_expressions:
        col_name, operator, filter_value = split_filter_part(filter_part)
        if operator in ('eq', 'ne', 'lt', 'le', 'gt', 'ge'):
            dff = dff.loc[getattr(dff[col_name], operator)(filter_value)]
        elif operator == 'contains':
            dff = dff.loc[dff[col_name].str.contains(filter_value)]
        elif operator == 'datestartswith':
            dff = dff.loc[dff[col_name].str.startswith(filter_value)]
    return dff.to_dict('records')

app.run_server(mode="inline")

<IPython.core.display.Javascript object>